In [62]:
import os
import pandas as pd
import numpy as np
ecg_data=pd.read_csv("datasets/ecg/ecg_dataset.csv")

In [63]:
#conversion of last row of characters into respective numeric values and storing in ecg_cat_1hot

ecg_cat=ecg_data.iloc[:,-1]
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
ecg_cat_1hot=encoder.fit_transform(ecg_cat)

#adding encoded to the original data
ecg_data.drop(ecg_data.columns[100], axis=1, inplace=True)
ecg_data[100]=ecg_cat_1hot

In [4]:
# stratified spliting of data to train and test

from sklearn.model_selection import StratifiedShuffleSplit
split= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(ecg_data, ecg_data.iloc[:,[-1]]):
    strat_train_set=ecg_data.loc[train_index]
    strat_test_set=ecg_data.loc[test_index]

In [4]:
#exporting data into csv

strat_train_set.to_csv(r"datasets/ecg/ecg_train.csv", index=None, header=True)
strat_test_set.to_csv(r"datasets/ecg/ecg_test.csv", index=None, header=True)

In [5]:
#data and label separation
y_train=strat_train_set.iloc[:,-1].copy()
x_train=strat_train_set.drop(strat_train_set.columns[100], axis=1)

In [6]:
#SGD CLASSIFIER cross-val score

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier(random_state=42, max_iter=5, tol=-np.infty)
cross_val_score(sgd_clf, x_train, y_train, cv=3, scoring="accuracy")

array([0.76239811, 0.76317367, 0.78088925])

In [7]:
#Random Forest cross-val score

from sklearn.ensemble import RandomForestClassifier
rand_forest=RandomForestClassifier(n_estimators=100)
cross_val_score(rand_forest, x_train, y_train, cv=3, scoring="accuracy")

array([0.96662443, 0.96648571, 0.96540267])

In [52]:
#Decision Tree cross-val score

from sklearn.tree import DecisionTreeClassifier
dec_tree=DecisionTreeClassifier()
cross_val_score(dec_tree, x_train, y_train, cv=3, scoring="accuracy")

array([0.93484143, 0.93482095, 0.93412804])

In [53]:
#sgd classifier training

sgd_clf.fit(x_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=-inf,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [57]:
#sgd scores on same train set

pred=sgd_clf.predict(x_train)
correct=sum(pred==y_train)
print(correct/len(pred))

0.7892621664050236


In [54]:
#random forest training

rand_forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [59]:
#random forest scores on same train set

pred=rand_forest.predict(x_train)
correct=sum(pred==y_train)
print(correct/len(pred))

1.0


In [55]:
#decision tree training

dec_tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [60]:
#decision tree scores on same train set

pred=dec_tree.predict(x_train)
correct=sum(pred==y_train)
print(correct/len(pred))

1.0


In [56]:
import pickle

#dump sgd classifier
pickle.dump(sgd_clf, open("Saved Models/sgd.sav", 'wb'))

#dump randomforest classifier
pickle.dump(rand_forest, open("Saved Models/randomforest.sav", 'wb'))

#dump decisiontree classifier
pickle.dump(dec_tree, open("Saved Models/decisiontree.sav", 'wb'))